# Visualize the Performance of the CBREE (vMFN) Method

In [8]:
from os import path
import rareeventestimation as ree
import pandas as pd
import plotly.express as px
from rareeventestimation.evaluation.constants import INDICATOR_APPROX_LATEX_NAME, BM_SOLVER_SCATTER_STYLE, MY_LAYOUT, DF_COLUMNS_TO_LATEX, LATEX_TO_HTML, WRITE_SCALE
import plotly.graph_objects as go
from IPython.display import display, Markdown
# recommended: use autoreload for development: https://ipython.readthedocs.io/en/stable/config/extensions/autoreload.html
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load Data
### Option 1: Get precomputed data online

In [9]:
# data is here: https://archive.org/details/konstantinalthaus-rareeventestimation-data
# you can got to this link and inspect the files pefore loading
df_agg = pd.read_json("https://ia801504.us.archive.org/23/items/konstantinalthaus-rareeventestimation-data/resampled_toy_problems.json")
df_bm_agg = pd.read_json("https://archive.org/download/konstantinalthaus-rareeventestimation-data/benchmark_toy_problems_aggregated.json")

### Option 2: Aggregate locally precomputed data

In [10]:
## uncomment to load existing data 
## or to compile data after computing it yourself:
# data_dir ="docs/benchmarking/data/cbree_sim/toy_problems_resampled"
# df_path =path.join(data_dir, "resampled_toy_problems.json")
# if not path.exists(df_path):
#     # load and clean
#     df = ree.load_data(data_dir, "*")
#     df.drop(columns=["index", "Unnamed: 0", "VAR Weighted Average Estimate","CVAR"], inplace=True, errors="ignore")
#     df.drop_duplicates(inplace=True)
#     df.reset_index(drop=True, inplace=True)
#     # Pretty names
#     to_drop = ["callback"] # no callbacks here
#     df.rename(columns={"mixture_model": "Method"}, inplace=True)
#     replace_values = {"Method": {"GM": "CBREE", "vMFNM": "CBREE (vMFN)"}}
#     df = df.drop(columns=to_drop) \
#         .rename(columns=DF_COLUMNS_TO_LATEX) \
#         .replace(replace_values)
#     # melt aggregated estimates into long format
#     df = df.rename(columns={"Estimate": "Last Estimate"})\
#         .melt(id_vars = [c for c in df.columns if not "Estimate" in c],
#               var_name="Averaging Method",
#               value_name="Estimate")
#     # make solver column with unique names wrt all options.
#     df = df.apply(expand_cbree_name, axis=1, columns= [DF_COLUMNS_TO_LATEX['observation_window'], "Averaging Method"])  
#     df = ree.add_evaluations(df,  only_success=True)
#     # %% aggregate
#     df_agg = ree.aggregate_df(df)
#     # save
#     df_agg.to_json(df_path)
# else:
#    df_agg = pd.read_json(df_path)
# # load benchmark
# df_bm, df_bm_agg = ree.get_benchmark_df()

## Make Figures

In [13]:
for prob in df_agg.Problem.unique():
    # filter
    this_df = df_agg.query("Problem == @prob & `Averaging Method`=='Average Estimate'")
    this_df = this_df[this_df['$N_{{ \\text{{obs}} }}$'].isin([2,5,10])]
    cmap = ree.sr_to_color_dict(this_df["$\\Delta_{{\\text{{Target}}}}$"].astype(float))
    this_df["cvar_tgt_str"] = this_df["$\\Delta_{{\\text{{Target}}}}$"].apply(str)
    this_df = this_df.sort_values(["$\\Delta_{{\\text{{Target}}}}$", "$N_{{ \\text{{obs}} }}$", "Sample Size"])
    # plot
    fig = px.line(
        this_df,
        x = "Relative Root MSE",
        y="Cost Mean",
        facet_col="Method",
        facet_row="$N_{{ \\text{{obs}} }}$",
        color_discrete_map=cmap,
        color="cvar_tgt_str",
        log_x=True,
        log_y=True,
        markers=True,
        hover_name="Sample Size",
        labels=LATEX_TO_HTML | {"cvar_tgt_str": LATEX_TO_HTML[DF_COLUMNS_TO_LATEX["cvar_tgt"]]})
    # add benchmark
    this_df_bm = df_bm_agg.query("Problem == @prob & cvar_tgt == 1")
    num_rows = len(this_df["$N_{{ \\text{{obs}} }}$"].unique())
    num_cols = len(this_df["Method"].unique())
    for bm_solver in this_df_bm.Solver.unique():
        dat =this_df_bm.query("Solver == @bm_solver")
        trace_dict = {
            "x" : dat["Relative Root MSE"],
            "y" : dat["Cost Mean"],
            "legendgrouptitle_text": "Benchmark Methods",
            "name": bm_solver,
            "legendgroup": "group",
            "mode": "markers+lines",
            "opacity": 0.8
        }
        trace_dict = trace_dict | BM_SOLVER_SCATTER_STYLE[bm_solver]
        fig = ree.add_scatter_to_subplots(fig, num_rows, num_cols, **trace_dict)
    # style
    fig.update_layout(**MY_LAYOUT)
    fig.update_layout(**{"width":700,
    "height":900})
    fig.for_each_annotation(
        lambda a: a.update(yshift =  -10 if a.text.startswith("Method") else 0)) # adjust column name position
    # show and save
    fig.show()
    fig.write_image(f"{prob} resampled stopping criterion.png".replace(" ", "_").lower(), scale=WRITE_SCALE)
    # make and save caption
    fig_description = f"Solving the {prob} with the CBREE methods using  \
    different parameters. \
    We vary the stopping criterion $\\Delta_{{\\text{{Target}}}}$ (color), \
    the divergence criterion $N_\\text{{obs}}$ (row) and \
    the method  (column). \
    The parameter $\\epsilon_{{\\text{{Target}}}} = {0.5}$ \
    and the choice of the indicator approximation {INDICATOR_APPROX_LATEX_NAME['algebraic']} \
    are fixed. \
    Furthermore we plot also the performance of the benchmark methods EnKF \
    and SiS. \
    Each marker represents the empirical estimates based the successful portion of $200$ simulations."
    with open(f"{prob} resampled stopping criterion desc.tex".replace(" ", "_").lower(), "w") as file:
        file.write(fig_description)
    display(Markdown(fig_description))

Solving the Fujita Rackwitz Problem (d=2) with the CBREE methods using      different parameters.     We vary the stopping criterion $\Delta_{\text{Target}}$ (color),     the divergence criterion $N_\text{obs}$ (row) and     the method  (column).     The parameter $\epsilon_{\text{Target}} = 0.5$     and the choice of the indicator approximation $I_\text{alg}$     are fixed.     Furthermore we plot also the performance of the benchmark methods EnKF     and SiS.     Each marker represents the empirical estimates based the successful portion of $200$ simulations.

Solving the Fujita Rackwitz Problem (d=50) with the CBREE methods using      different parameters.     We vary the stopping criterion $\Delta_{\text{Target}}$ (color),     the divergence criterion $N_\text{obs}$ (row) and     the method  (column).     The parameter $\epsilon_{\text{Target}} = 0.5$     and the choice of the indicator approximation $I_\text{alg}$     are fixed.     Furthermore we plot also the performance of the benchmark methods EnKF     and SiS.     Each marker represents the empirical estimates based the successful portion of $200$ simulations.

Solving the Linear Problem (d=2) with the CBREE methods using      different parameters.     We vary the stopping criterion $\Delta_{\text{Target}}$ (color),     the divergence criterion $N_\text{obs}$ (row) and     the method  (column).     The parameter $\epsilon_{\text{Target}} = 0.5$     and the choice of the indicator approximation $I_\text{alg}$     are fixed.     Furthermore we plot also the performance of the benchmark methods EnKF     and SiS.     Each marker represents the empirical estimates based the successful portion of $200$ simulations.

Solving the Linear Problem (d=50) with the CBREE methods using      different parameters.     We vary the stopping criterion $\Delta_{\text{Target}}$ (color),     the divergence criterion $N_\text{obs}$ (row) and     the method  (column).     The parameter $\epsilon_{\text{Target}} = 0.5$     and the choice of the indicator approximation $I_\text{alg}$     are fixed.     Furthermore we plot also the performance of the benchmark methods EnKF     and SiS.     Each marker represents the empirical estimates based the successful portion of $200$ simulations.